In [24]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import math
'''DATA'''

# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# For this example, the output y is a linear function of (x, x^2, x^3), so
# we can consider it as a linear layer neural network. Let's prepare the
# tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# In the above code, x.unsqueeze(-1) has shape (2000, 1), and p has shape
# (3,), for this case, broadcasting semantics will apply to obtain a tensor
# of shape (2000, 3) 
train_ds = TensorDataset(xx, y)
val_ds = TensorDataset(xx, y)
train_ds[0]
from sklearn.model_selection import train_test_split
'''DATASETS'''

x_train, x_test, y_train, y_test = train_test_split(
    xx, y, test_size=0.33, random_state=42
)

bs = 32
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_test, y_test)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)


In [29]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

"""DATASETS"""
x, y = load_iris(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=42
)

bs = 32
train_ds = TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(torch.from_numpy(x_test), torch.from_numpy(y_test))
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)
x[0], y[0]

(array([5.1, 3.5, 1.4, 0.2]), 0)

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class NN(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.vocab_size = n
        self.embedding_dim = 2
        self.l1 = nn.Linear(4,3)
        self.embed = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.flat = nn.Flatten(0,1)

    def forward(self, x):
        x = self.l1(x)
        x = F.sigmoid(x)
        return self.flat(x)

In [27]:
def train(train_dl, valid_dl, model, epochs):
    learning_rate = 1e-2

    optimizer = torch.optim.Adam(
        model.parameters(), lr=learning_rate, weight_decay=1e-5
    )
    history = []
    early_stop = 0
    for e in range(epochs):
        model.train()
        for x, y in train_dl:
            # Forward
            pred = model(x)

            # Compute loss
            # Note that you should only compute the losses of the nodes in the training set.

            loss = F.mse_loss(pred, y)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # end batch

        model.eval()
        with torch.no_grad():
            valid_loss = sum(F.mse_loss(model(xb), yb) for xb, yb in valid_dl)
            valid_loss = valid_loss / len(valid_dl)

        history.append(
            [
                loss.item(),
                valid_loss.item(),
            ]
        )

        if e % 100 == 0:
            val_acc = 0
            print(f"Epoch {e}, loss: {loss:.3f}, valid_loss {valid_loss:.3f}")

    return history


# model = GCN(g.ndata["x"].shape[1], 64, g.ndata["label"].unique().shape[0])
model = NN(3)
history = train(train_dl, valid_dl, model, epochs=10)


TypeError: __init__() missing 1 required positional argument: 'n'

In [ ]:
import pandas as pd
df = pd.DataFrame(history, columns=['loss','valid_loss'])
print(df.dtypes)
df[['loss','valid_loss']].plot()
df.head()

In [ ]:
import matplotlib.pyplot as plt

model.eval()
p = model(xx)

fig, ax = plt.subplots()
ax.plot(xx[:, 0], p.detach().numpy(), label='pred')
ax.plot(xx[:, 0], y, label='y')
plt.legend()
